In [1]:
# %load_ext autoreload
# %autoreload 2

import sys

sys.path.insert(0, "../")

from autogluon.vision import ImagePredictor, ImageDataset
import numpy as np
import pandas as pd
import pickle
import datetime
from pathlib import Path
import cleanlab
from cross_validation_autogluon import cross_val_predict_autogluon_image_dataset, train_predict_autogluon

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

/home/ubuntu/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read original data

In [2]:
# data path
CIFAR_10_DATA_PATH = "./cifar10_png/"

# read data from root folder
train_dataset, _, test_dataset = \
    ImageDataset.from_folders(
        root=CIFAR_10_DATA_PATH,
    )

## Read data with noisy label

In case cross-validation procedure fails for a model, we want to be able to rerun with the same noisy dataset

In [3]:
noise_amount = 0.2
frac_zero_noise_rates = 0.4

# DATA_NOISY_LABELS_OUT_FILE = f"cifar10_train_dataset_noise_amount_{noise_amount}_sparsity_{frac_zero_noise_rates}_20220326055753.csv"
DATA_NOISY_LABELS_OUT_FILE = f"cifar10_train_dataset_noise_amount_{noise_amount}_sparsity_{frac_zero_noise_rates}_20221011184524.csv"
train_dataset_noisy_labels_loaded = pd.read_csv(DATA_NOISY_LABELS_OUT_FILE)
# train_dataset_noisy_labels_loaded['image'] = train_dataset_noisy_labels_loaded.apply(lambda x: 
#                                                                                      CIFAR_10_DATA_PATH + x['image'][18:],axis=1)

In [4]:
train_dataset_noisy_labels_loaded.head()

,image,label
0,/home/ubuntu/label-error-detection-benchmarks/src/experiments/cifar-10/cifar10_png/train/airplane/data_batch_1_index_0029.png,0
1,/home/ubuntu/label-error-detection-benchmarks/src/experiments/cifar-10/cifar10_png/train/airplane/data_batch_1_index_0030.png,5
2,/home/ubuntu/label-error-detection-benchmarks/src/experiments/cifar-10/cifar10_png/train/airplane/data_batch_1_index_0035.png,9
3,/home/ubuntu/label-error-detection-benchmarks/src/experiments/cifar-10/cifar10_png/train/airplane/data_batch_1_index_0049.png,7
4,/home/ubuntu/label-error-detection-benchmarks/src/experiments/cifar-10/cifar10_png/train/airplane/data_batch_1_index_0077.png,8


In [5]:
train_dataset_noisy_labels_loaded.shape

(50000, 2)

In [6]:
train_dataset_noisy_labels_loaded.groupby("label")["image"].count().reset_index()

,label,image
0,0,3690
1,1,5282
2,2,3693
3,3,5883
4,4,5962
5,5,4411
6,6,5275
7,7,5790
8,8,6030
9,9,3984


## Generate In-Sample Predicted Probabilities

In [7]:
%%time

models = [
#     "resnet18",
#     "resnet50d",
#     "efficientnet_b1",
    "twins_pcpvt_base",
    "swin_base_patch4_window7_224"
]

epochs = 100
holdout_frac = 0.2
n_splits = 5

# run cross-validation for each model
for model in models:
    
    print("----")
    print(f"Running cross-validation for model: {model}")

    MODEL_PARAMS = {
        "model": model,
        "epochs": epochs,
        "holdout_frac": holdout_frac
    }
    out_folder = f"./noxval_cifar10_train_dataset_noise_amount_{noise_amount}_sparsity_{frac_zero_noise_rates}_cv_{model}/"
   
    _ = \
    train_predict_autogluon(
        dataset=train_dataset_noisy_labels_loaded, # train with NOISY LABELS,
        classes=train_dataset.classes,
        out_folder=out_folder, # save results of cross-validation in pickle files for each fold
        n_splits=n_splits,
        model_params=MODEL_PARAMS,
        time_limit=30,
    )
    
    images = np.load(out_folder + "images.npy", allow_pickle=True)
    label_name_to_idx_map = {v:k for k, v in enumerate(train_dataset.classes)}
    get_orig_label_idx_from_file_path = np.vectorize(lambda f: label_name_to_idx_map[Path(f).parts[-2]])
    true_labels = get_orig_label_idx_from_file_path(images)
    np.save(out_folder + "true_labels", true_labels)

modified configs(<old> != <new>): {
root.img_cls.model   resnet101 != twins_pcpvt_base
root.misc.num_workers 4 != 8
root.misc.seed       42 != 259
root.train.early_stop_baseline 0.0 != -inf
root.train.epochs    200 != 100
root.train.early_stop_patience -1 != 10
root.train.early_stop_max_value 1.0 != inf
root.train.batch_size 32 != 16
}
Saved config to /home/ubuntu/label-error-detection-benchmarks/src/experiments/cifar-10/7897411c/.trial_0/config.yaml


----
Running cross-validation for model: twins_pcpvt_base
training...


Model twins_pcpvt_base created, param count:                                         43320586
AMP not enabled. Training in float32.
Disable EMA as it is not supported for now.
Start training from [Epoch 0]
Epoch[0] Batch [49]	Speed: 101.125628 samples/sec	accuracy=0.133750	lr=0.000100
Epoch[0] Batch [99]	Speed: 117.254976 samples/sec	accuracy=0.166250	lr=0.000100
Epoch[0] Batch [149]	Speed: 116.736547 samples/sec	accuracy=0.188750	lr=0.000100
Epoch[0] Batch [199]	Speed: 116.848909 samples/sec	accuracy=0.226250	lr=0.000100
`time_limit=29.999898672103882` reached, exit early...


predicting...
Saving to numpy files in this folder: ./noxval_cifar10_train_dataset_noise_amount_0.2_sparsity_0.4_cv_twins_pcpvt_base/
saving pred_probs...
saving noisy_labels...
saving images...
saving indices...
saving predictor...


modified configs(<old> != <new>): {
root.img_cls.model   resnet101 != swin_base_patch4_window7_224
root.misc.num_workers 4 != 8
root.misc.seed       42 != 136
root.train.early_stop_baseline 0.0 != -inf
root.train.epochs    200 != 100
root.train.early_stop_patience -1 != 10
root.train.early_stop_max_value 1.0 != inf
root.train.batch_size 32 != 16
}
Saved config to /home/ubuntu/label-error-detection-benchmarks/src/experiments/cifar-10/468e481c/.trial_0/config.yaml


----
Running cross-validation for model: swin_base_patch4_window7_224
training...


Model swin_base_patch4_window7_224 created, param count:                                         86753474
AMP not enabled. Training in float32.
Disable EMA as it is not supported for now.
Start training from [Epoch 0]
Epoch[0] Batch [49]	Speed: 73.995279 samples/sec	accuracy=0.168750	lr=0.000100
Epoch[0] Batch [99]	Speed: 78.905585 samples/sec	accuracy=0.205625	lr=0.000100
Epoch[0] Batch [149]	Speed: 78.762440 samples/sec	accuracy=0.243750	lr=0.000100
`time_limit=29.999879598617554` reached, exit early...


predicting...
Saving to numpy files in this folder: ./noxval_cifar10_train_dataset_noise_amount_0.2_sparsity_0.4_cv_swin_base_patch4_window7_224/
saving pred_probs...
saving noisy_labels...
saving images...
saving indices...
saving predictor...
CPU times: user 6min 9s, sys: 1min 1s, total: 7min 10s
Wall time: 7min 10s


In [8]:
train_dataset_noisy_labels_loaded["label"].unique()

array([0, 5, 9, 7, 8, 1, 2, 4, 6, 3])

## Display GPU used for training

In [9]:
!nvidia-smi

Tue Oct 11 23:00:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.08    Driver Version: 510.73.08    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   66C    P0    62W / 300W |   6225MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------